# Imports

In [1]:
import numpy as np
import os
from random import randint as rand
import cv2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Model preparation 

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

## Load a (frozen) Tensorflow model into memory.

In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Detection

In [4]:
def run_inference_for_single_image(image_np, graph):
    with graph.as_default():
        with tf.Session() as sess:
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            detection_boxes = tf.get_default_graph().get_tensor_by_name('detection_boxes:0')
            detection_scores = tf.get_default_graph().get_tensor_by_name('detection_scores:0')
            detection_classes = tf.get_default_graph().get_tensor_by_name('detection_classes:0')
            num_detections = tf.get_default_graph().get_tensor_by_name('num_detections:0')

            (boxes, scores, classes, num) = sess.run([
                detection_boxes, 
                detection_scores,
                detection_classes, 
                num_detections
            ], 
                feed_dict={image_tensor: image_np_expanded}
            )
            
            output_dict = {}
            
            output_dict['num_detections'] = int(np.squeeze(num))
            output_dict['detection_classes'] = np.squeeze(classes).astype(np.uint8)
            output_dict['detection_boxes'] = np.squeeze(boxes)
            output_dict['detection_scores'] = np.squeeze(scores)

    return output_dict

In [5]:
def get_index_labels_dict():
    with open('mscoco_label_map.pbtxt', 'r') as f:
        labels = f.readlines()
        labels = [i.strip() for i in labels] 
        
    index_labels_dict = {}
    i = 0
    while i+4 <= len(labels)-1:
        ind, name = None, None
        for j in range(i, i+5):
            s = labels[j].split()
            if 'id:' in s:
                ind = int(s[-1])
            elif 'display_name:' in s:
                name = s[-1][1:-1]

        if ind and name: index_labels_dict[ind] = name
        i = j+1
    return index_labels_dict

In [6]:
index_labels_dict = get_index_labels_dict()

def draw_box_on_image(scores, boxes, classes, im_width, im_height, image_np,):
    index_to_colors = {i: (rand(0, 255), rand(0, 255), rand(0, 255)) for i in index_labels_dict}
    
    for i in range(100):
        if scores[i] > 0.65:
            (left, right, top, bottom) = (
                boxes[i][1] * im_width, 
                boxes[i][3] * im_width,
                boxes[i][0] * im_height, 
                boxes[i][2] * im_height
            )
            
            p1 = (int(left), int(top))
            p2 = (int(right), int(bottom))

            cv2.rectangle(
                image_np, 
                p1, 
                p2, 
                index_to_colors[int(classes[i])] , 
                3, 
                1
            )

            cv2.putText(
                image_np, 
                index_labels_dict[int(classes[i])] + ": " + str(scores[i])[2:4] + "%", 
                (int(left), int(top)-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.8, 
                index_to_colors[int(classes[i])], 
                2
            )


# Detection on Image

In [10]:
img = cv2.imread('image.jpg')
h, w = img.shape[:2]
img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
output_dict = run_inference_for_single_image(img, detection_graph)
im_height, im_width = img.shape[:2]
draw_box_on_image(output_dict['detection_scores'], output_dict['detection_boxes'], output_dict['detection_classes'], im_width, im_height, img)

cv2.imshow('graycsale image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Detection on Video

In [7]:
cap = cv2.VideoCapture('car1.mp4')
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('carop.mp4', fourcc, 29.97, (width,  height))

while cap.isOpened():
    ret, image_np = cap.read()
    if not ret:
        break
    
    image_np = cv2.resize(image_np, (width, height), interpolation=cv2.INTER_AREA)
    im_height, im_width = image_np.shape[:2]

    output_dict = run_inference_for_single_image(image_np, detection_graph)
    draw_box_on_image(
        output_dict['detection_scores'], 
        output_dict['detection_boxes'], 
        output_dict['detection_classes'], 
        im_width, 
        im_height, 
        image_np
    )
    
    out.write(image_np)
    cv2.imshow('object_detection', image_np)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
